In [23]:
# Outside packages
import numpy as np

# Bokeh packages
from bokeh.io import output_notebook, show
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.models.widgets import Slider
from bokeh.plotting import figure

In [24]:
output_notebook()

Loading BokehJS ...

In [91]:
# + + + Set Up Default View of Model + + +
# Fixed Parameters
Vm        = 1    # Maximum growth rate (per day)
Kn        = 1    # Half-saturation constant for nitrogen uptake (umolN per l)
Rm        = 1    # Maximum grazing rate (per day)
g         = 0.2; # Zooplankton death rate (per day)
lambda_Z  = 0.2  # Grazing constant (umolN per l)
epsilon   = 0.1  # Phyto death rate (per day)
gamma_Z   = 0.7  # Dimensionless proportion of assimilated nitrogen by Zooplankton
f         = 0.25 # Light intensity (assumed constant)
dt        = 1    # Timestep of 1 day

# Initial Conditions
N_0 = 4
P_0 = 2.5
Z_0 = 0.5

# Initialize Arrays
N = np.empty(200, dtype="float")
P = np.empty(200, dtype="float")
Z = np.empty(200, dtype="float")

# Insert Initial Values
N[0] = N_0
P[0] = P_0
Z[0] = Z_0

In [92]:
# + + + Run Default model + + +
for idx in np.arange(1, 200, 1):
    t = idx - 1
    
    # Common terms
    gamma_N   = N[t] / (Kn + N[t])
    zoo_graze = Rm * (1 - np.exp(-lambda_Z * P[t])) * Z[t]
    
    # Equation calculations
    N[idx] = dt * (-Vm*gamma_N*f*P[t] + (1-gamma_Z)*zoo_graze + epsilon*P[t] + g*Z[t]) + N[t]
    P[idx] = P[t]*(1 - epsilon*dt + Vm*gamma_N*f*dt) - (zoo_graze * dt);
    Z[idx] = dt * (gamma_Z*zoo_graze - g*Z[t]) + Z[t]   

In [93]:
# Initial Datapoints
x = np.arange(1, 201, 1)
N = N
P = P
Z = Z

# Always put into a Bokeh data source object
source = ColumnDataSource(data = {
        'x' : x,
        'N' : N,
        'P' : P,
        'Z' : Z,
    })

In [94]:
# Functions for plotting
def plotlines(plot, x, y, source, line_width=2, line_alpha=0.6,
             color='black'):
    plot.line(x, y, source=source, line_width=line_width,
             line_alpha=line_alpha, color=color)

In [95]:
plot = figure(plot_width=900, plot_height=300)
plotlines(plot, 'x', 'N', source, color="firebrick")
plotlines(plot, 'x', 'P', source, color="navy")
plotlines(plot, 'x', 'Z', source, color="black")

callback = CustomJS(args=dict(source=source), code="""
    // This part is key... this is  the data you need to ingest to modify.
    var data = source.get('data');
    x = data['x'];
    N = data['N'];
    P = data['P'];
    Z = data['Z'];
    
    // Fixed Parameters
    var Vm = 1;
    var Kn = 1;
    var Rm = 1;
    var g = 0.2;
    var lambda_Z = 0.2;
    var epsilon = 0.1;
    var gamma_Z = 0.7;
    var f = 0.25;
    var dt = 1;
    
    // Initial Conditions
    var N_0 = nut.get('value');
    var P_0 = phyto.get('value');
    var Z_0 = zoo.get('value');
    
    // Insert Initial Values
    N[0] = N_0
    P[0] = P_0
    Z[0] = Z_0

    // Run Model
    for (i = 1; i < x.length; i++) {
         t = i - 1;

        // Common terms
        gamma_N   = N[t] / (Kn + N[t])
        zoo_graze = Rm * (1 - Math.exp(-lambda_Z * P[t])) * Z[t]
    
        // Equation calculations
        N[i] = dt * (-Vm*gamma_N*f*P[t] + (1-gamma_Z)*zoo_graze + epsilon*P[t] + g*Z[t]) + N[t]
        P[i] = P[t]*(1 - epsilon*dt + Vm*gamma_N*f*dt) - (zoo_graze * dt);
        Z[i] = dt * (gamma_Z*zoo_graze - g*Z[t]) + Z[t]   
    }
    source.trigger('change');
""")

# With multiple widgets, need to set a name for when calling them.

# Nutrient
nut_slider = Slider(start = 0, end = 10, value = 4, step = 0.1, title = "Initial Nutrient Concentration", callback=callback)
callback.args["nut"] = nut_slider

# Phyto
phyto_slider = Slider(start = 0, end = 10, value = 2.5, step = 0.1, title = "Initial Phytoplankton Concentration", callback=callback)
callback.args["phyto"] = phyto_slider

# Zoo
zoo_slider = Slider(start = 0, end = 10, value = 0.5, step = 0.1, title = "Initial Zooplankton Concentration", callback=callback)
callback.args["zoo"] = zoo_slider

layout = column(nut_slider, phyto_slider, zoo_slider, plot)

show(layout)